In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


In [22]:

data = pd.read_csv("/content/drive/MyDrive/housing.new.csv")

data.fillna(method='ffill', inplace=True)


<ipython-input-22-95293d105565>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)


In [20]:
X = data[["longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms", "population", "households", "median_income"]]
y = data["median_house_value"]



In [23]:
categorical_features = ["housing_median_age"]  # Assuming this is categorical
encoder = OneHotEncoder(handle_unknown='ignore')
X_encoded = pd.concat([X, pd.DataFrame(encoder.fit_transform(X[categorical_features]).toarray(), columns=encoder.get_feature_names_out(categorical_features))], axis=1)
X_encoded.drop(categorical_features, axis=1, inplace=True)


In [19]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_encoded)


In [24]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.5, random_state=42)


In [14]:
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor()
}

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f"{model.__class__.__name__} - MSE: {mse:.2f}, R^2: {r2:.2f}")


In [15]:
for name, model in models.items():
    model.fit(X_train, y_train)
    evaluate_model(model, X_test, y_test)


LinearRegression - MSE: 5044245660.01, R^2: 0.62
RandomForestRegressor - MSE: 2350872453.13, R^2: 0.82


In [17]:
param_grid = {
    "n_estimators": [100, 150],
    "max_depth": [5, 7],
    "min_samples_split": [5, 10]
}

random_search = RandomizedSearchCV(RandomForestRegressor(), param_grid, n_iter=5, cv=5, scoring="neg_mean_squared_error")
from joblib import parallel_backend

with parallel_backend('threading'):
    random_search.fit(X_train, y_train)
best_model = random_search.best_estimator_


In [25]:
evaluate_model(best_model, X_test, y_test)


RandomForestRegressor - MSE: 3640890640.24, R^2: 0.73


In [28]:
#UNSEEN DATA
new_data = pd.DataFrame({
    "longitude": [122.41, 121.88, 122.49],
    "latitude": [37.76, 37.63, 37.73],
    "housing_median_age": [30, 15, 22],
    "total_rooms": [1200, 800, 950],
    "total_bedrooms": [3, 2, 3],
    "population": [3500, 2000, 3000],
    "households": [1500, 800, 1200],
    "median_income": [4.5, 6.0, 5.5]
})

print(new_data)


   longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0     122.41     37.76                  30         1200               3   
1     121.88     37.63                  15          800               2   
2     122.49     37.73                  22          950               3   

   population  households  median_income  
0        3500        1500            4.5  
1        2000         800            6.0  
2        3000        1200            5.5  


In [31]:
new_data_encoded = pd.concat([new_data, pd.DataFrame(encoder.transform(new_data[categorical_features]).toarray(), columns=encoder.get_feature_names_out(categorical_features))], axis=1)
new_data_encoded.drop(categorical_features, axis=1, inplace=True)
new_data_scaled = scaler.transform(new_data_encoded)

predictions = best_model.predict(new_data_scaled)
rounded_predictions = [round(price, 3) for price in predictions]
print("Predicted house prices (rounded):", rounded_predictions)



Predicted house prices (rounded): [153853.103, 220043.309, 203242.505]
